---
title: Global Variables
order: 1
---

This topic confused me for a long time and I basically never used this concept to my advantage, but only because I didn't understand one simple thing: Any variable defined at a module level is a global variable.

In this article we will discuss two python keywords - `global` and `nonlocal` - and scoping.

## Terminology

A global scope is the one at the level of a module. That is also the level that your IPython shell is in when you start a new one. Any variable you define at this level is a *global variable*.

Inside this scope you can create an inner scope, for example by defining a function. And now things start to be interesting. Let's assume we are in the scope of this function. Now, the current scope is called *local* scope and the outer scope (which also happens to be the global scope in this case) is called *nonlocal* scope.

This terminology holds for any number of nested scopes: The current scope is *local*, the closest higher scope is *nonlocal* and the top-most scope is *global*.

## Scopes in Python

There is surprisingly one scope above global - the *built-in* scope. The full list of scopes in Python is this:

- built-in
- global
- nonlocal (recursive)
- local

This means that when a variable is referenced in a local scope, Python first looks for it in the local scope. If it isn't found there the nearest enclosing scope is searched (the nonlocal scope). If it is not found there then the next higher scope is searched and so on, until a global scope is reached. If a variable is not found in the global scope, the built-in scope is searched. If the variable is not even here, a NameError is raised.

## Nesting of Scopes

It is important to remind here that nested scopes are created by *defining* new functions (or classes etc., see below), NOT by calling them. For example, these are not nested scopes, the function `innerfn` isn't actually "inner" and the function `outerfn` isn't actually "outer". They are same-level functions, just called from one another. They do not create nested scopes.

In [ ]:
def innerfn():
    print(x)

def outerfn():
    x = 2
    innerfn()

x = 1
outerfn()

1


We can see it because when `innerfn` is called from within `outerfn`, the number $1$ is printed, and not $2$. This is what happened:

- First, two functions are defined in the global scope.
- Then, the variable `x` is assigned the value 1 and the `outerfn` function is called.
- In the scope of the `outerfn` function, the local variable `x` is defined and assigned the value 2.
- The `innerfn` function is called in the scope of `outerfn`.
- The `innerfn` function, upon calling, creates its own scope (third so far).
- The variable `x` is referenced in this bottom-most level.
- It is not found there, so Python goes search the scope within which `innerfn` is defined (which is the global scope here). The middle scope (the scope of `outerfn`) is completely skipped. Yes, `innerfn` is called from here, but that doesn't matter.
- The scope in which `innerfn` is defined - the global scope - does contain the variable `x`, so it is found and the value $2$ is used.
- The value $2$ is printed, `innerfn` returns and its scope pops.
- `outerfn` returns and its scope pops.

In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                def fn5():
                    x = 5
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 5
inside fn4: x = 4
inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                def fn5():
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 4
inside fn4: x = 4
inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            def fn4():
                def fn5():
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 2
inside fn4: x = 2
inside fn3: x = 2
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                def fn5():
                    global x
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 0
inside fn4: x = 4
inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                def fn5():
                    nonlocal x
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 4
inside fn4: x = 4
inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                def fn5():
                    x = 5
                    nonlocal x
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

SyntaxError: name 'x' is assigned to before nonlocal declaration (1197466916.py, line 12)

In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            nonlocal x
            def fn4():
                nonlocal x
                def fn5():
                    nonlocal x
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 2
inside fn4: x = 2
inside fn3: x = 2
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            def fn4():
                def fn5():
                    nonlocal x
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn5: x = 2
inside fn4: x = 2
inside fn3: x = 2
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            def fn4():
                x = 4
                print(f'inside start of fn4: {x = }')
                def fn5():
                    nonlocal x
                    x = 8
                    print(f'inside fn5: {x = }')
                fn5()
                print(f'inside end of fn4: {x = }')
            fn4()
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside start of fn4: x = 4
inside fn5: x = 8
inside end of fn4: x = 8
inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


In [ ]:
def fn():
    global y
    y = 2
fn()
y

2

In [ ]:
def fn():
    global y
fn()
y

NameError: name 'y' is not defined